In [1]:
!pip install sklearn_crfsuite
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install tensorflow==1.15.0 keras==2.2.4
!pip3 install glove-python-binary
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 747kB 27.5MB/s 
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-jjluljp6
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-jjluljp6
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=828cec8f6f5098b2f200cc759539aeb6d3285b4daec31b996e37b59d958300be
  Stored in directory: /tmp/pip-ephem-wheel-cache-cslh213u/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 317kB 34.5MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 512kB 59.9MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=a78c655d402bdfd498aa4ac71487f23e5a25b80db4e2721b872a7515

In [2]:
!gdown --id 1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_
!gdown --id 1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD
!gdown --id 1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s

Downloading...
From: https://drive.google.com/uc?id=1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_
To: /content/NER_RFQ_agg.csv
4.39MB [00:00, 39.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD
To: /content/NER_RFQ_agg_train.csv
3.48MB [00:00, 55.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s
To: /content/NER_RFQ_agg_test.csv
100% 909k/909k [00:00<00:00, 14.4MB/s]


In [3]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import datetime
import os
import numpy as np

from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf

import gensim.downloader as api

Using TensorFlow backend.


In [4]:
df = pd.read_csv('NER_RFQ_agg.csv', converters={'tokens': eval, 'ner_tags': eval})
train = pd.read_csv('NER_RFQ_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_RFQ_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

all_dfs = [df, train, test]

In [ ]:
all_tags = set()
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
label_list = {i: t for i, t in enumerate(all_tags)}
labels = list(tag_to_idx.keys())
labels.remove('O')
labels = sorted(labels, key=lambda name: (name[1:], name[0]))

In [5]:
pretrained_w2v = api.load("word2vec-ruscorpora-300")
pretrained_w2v.save_word2vec_format('tmp')

[==================================================] 100.0% 198.8/198.8MB downloaded


In [7]:
def add_suffix(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in pretrained_w2v.vocab.keys():
                res.append(w + suffix)
                has_found = True
                break
        if not has_found:
            res.append(w)
    return res

for d in all_dfs:
    d['tokens_with_suffix'] = d.apply(add_suffix, axis=1)

In [8]:
from gensim.models import Word2Vec

w2v = Word2Vec(sentences=df['tokens_with_suffix'], size=300, window=5, min_count=2)

In [9]:
w2v.intersect_word2vec_format('tmp')

In [11]:
def find_token(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ', '']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in w2v.wv.vocab.keys():
                res.append(w2v.wv.vocab[w + suffix].index)
                has_found = True
                break
        if not has_found:
            res.append(0)
    return res

for d in all_dfs:
    d['encoded_ner_tags'] = d.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
    d['int_tokens'] = d.apply(find_token, axis=1)

In [12]:
df

,id,tokens,ner_tags,tokens_with_suffix,encoded_ner_tags,int_tokens
0,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 119287r.msg, <aogdesk@nordstar.ru>...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 0, 206, 744, 293, 22, 190, 0, 30, 36, 237..."
1,119735R.msg,"[name, 119735r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 119735r.msg, <aogdesk@nordstar.ru>...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 0, 206, 744, 293, 22, 190, 6707, 624, 106..."
2,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good...","[name_NOUN, 120421r.msg, d.klebcha@s7.ru, <d.k...","[2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, ...","[19, 0, 5524, 5525, 5526, 2961, 22, 57, 874, 2..."
3,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 120660r.msg, yuliya, a., kondratov...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 5530, 846, 833, 518, 847, 12, 22, 57, 255..."
4,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 120660r.msg, yuliya, a., kondratov...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 5530, 846, 833, 518, 847, 12, 22, 57, 255..."
...,...,...,...,...,...,...
1187,123527R.msg,"[name, 123527r.msg, =?utf-8?b?0kprincw0lrqvtcy...","[O, O, O, O, O, O, O, O, B-GoodsString, O, O, ...","[name_NOUN, 123527r.msg, =?utf-8?b?0kprincw0lr...","[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, ...","[19, 0, 1249, 1250, 1251, 12, 22, 679, 0, 1026..."
1188,123088R.msg,"[name, 123088r.msg, lakshmi, suresh, <lakshmi@...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 123088r.msg, lakshmi, suresh, <lak...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 6241, 4264, 4265, 6242, 2961, 22, 147, 62..."
1189,123508R.msg,"[name, 123508r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 123508r.msg, <aogdesk@nordstar.ru>...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 7935, 206, 744, 293, 22, 190, 7936, 30, 3..."
1190,124894R.msg,"[name, 124894r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[name_NOUN, 124894r.msg, <anton.peshko@utair.r...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[19, 0, 304, 47, 305, 304, 22, 115, 5523, 278,..."


In [13]:
import warnings

warnings.filterwarnings("ignore")

In [11]:
def focal_loss(gamma=2., alpha=1.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

## PRETRAINED W2V(not fixed)

In [21]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3004, 300)         2544900   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 3004, 200)         320800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3004, 3)           603       
_________________________________________________________________
crf_3 (CRF)                  (None, 3004, 3)           27        
Total params: 2,866,330
Trainable params: 2,866,330
Non-trainable params: 0
_________________________________________________________________


In [22]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='pretrained_best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('pretrained_best_model.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 94s 110ms/step - loss: 0.6816 - crf_marginal_accuracy: 0.8232 - val_loss: 0.5591 - val_crf_marginal_accuracy: 0.9385
Epoch 2/30
857/857 [==============================] - 91s 106ms/step - loss: 0.5166 - crf_marginal_accuracy: 0.5827 - val_loss: 0.4881 - val_crf_marginal_accuracy: 0.0336
Epoch 3/30
857/857 [==============================] - 91s 106ms/step - loss: 0.4625 - crf_marginal_accuracy: 0.5531 - val_loss: 0.4596 - val_crf_marginal_accuracy: 0.9944
Epoch 4/30
857/857 [==============================] - 93s 109ms/step - loss: 0.4430 - crf_marginal_accuracy: 0.9971 - val_loss: 0.4523 - val_crf_marginal_accuracy: 0.9968
Epoch 5/30
857/857 [==============================] - 91s 107ms/step - loss: 0.4267 - crf_marginal_accuracy: 0.9974 - val_loss: 0.4438 - val_crf_marginal_accuracy: 0.9969
Epoch 6/30
857/857 [==============================] - 92s 108ms/step - loss: 0.3968 - crf_marginal_a

In [23]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.9153    0.8377    0.8748      1109
I-GoodsString     0.9250    0.8038    0.8602      1718

    micro avg     0.9211    0.8171    0.8660      2827
    macro avg     0.9201    0.8208    0.8675      2827
 weighted avg     0.9212    0.8171    0.8659      2827



## PRETRAINED W2V(fixed)

In [31]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 20

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3004, 300)         2544900   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 3004, 40)          51360     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 3004, 3)           123       
_________________________________________________________________
crf_7 (CRF)                  (None, 3004, 3)           27        
Total params: 2,596,410
Trainable params: 51,510
Non-trainable params: 2,544,900
_________________________________________________________________


In [32]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='pretrained_best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('pretrained_best_model_fixed_embeds.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 92s 107ms/step - loss: 0.5753 - crf_marginal_accuracy: 0.9856 - val_loss: 0.4858 - val_crf_marginal_accuracy: 0.0375
Epoch 2/30
857/857 [==============================] - 87s 101ms/step - loss: 0.4676 - crf_marginal_accuracy: 0.5657 - val_loss: 0.4711 - val_crf_marginal_accuracy: 0.9963
Epoch 3/30
857/857 [==============================] - 86s 101ms/step - loss: 0.4607 - crf_marginal_accuracy: 0.9918 - val_loss: 0.4660 - val_crf_marginal_accuracy: 0.9772
Epoch 4/30
857/857 [==============================] - 87s 101ms/step - loss: 0.4562 - crf_marginal_accuracy: 0.9901 - val_loss: 0.4625 - val_crf_marginal_accuracy: 0.9962
Epoch 5/30
857/857 [==============================] - 87s 102ms/step - loss: 0.4548 - crf_marginal_accuracy: 0.9966 - val_loss: 0.4610 - val_crf_marginal_accuracy: 0.9958
Epoch 6/30
857/857 [==============================] - 89s 103ms/step - loss: 0.4533 - crf_marginal_a

In [33]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.3455    0.1190    0.1771      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.3455    0.0467    0.0823      2827
    macro avg     0.1728    0.0595    0.0885      2827
 weighted avg     0.1356    0.0467    0.0695      2827



## W2V(not fixed, window=5, min_count=2) + BILSTM + CRF

In [36]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=5, min_count=2)
for d in all_dfs:
    d['int_tokens'] = d.apply(find_token, axis=1)

In [40]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 50

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 3004, 300)         2544900   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 3004, 100)         140400    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 3004, 3)           303       
_________________________________________________________________
crf_11 (CRF)                 (None, 3004, 3)           27        
Total params: 2,685,630
Trainable params: 2,685,630
Non-trainable params: 0
_________________________________________________________________


In [41]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='not_pretrained_best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('not_pretrained_best_model.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 98s 114ms/step - loss: 0.4957 - crf_marginal_accuracy: 0.6872 - val_loss: 0.4646 - val_crf_marginal_accuracy: 0.9956
Epoch 2/30
857/857 [==============================] - 92s 107ms/step - loss: 0.4535 - crf_marginal_accuracy: 0.4771 - val_loss: 0.4569 - val_crf_marginal_accuracy: 0.0709
Epoch 3/30
857/857 [==============================] - 91s 107ms/step - loss: 0.4468 - crf_marginal_accuracy: 0.8567 - val_loss: 0.4542 - val_crf_marginal_accuracy: 0.9965
Epoch 4/30
857/857 [==============================] - 91s 107ms/step - loss: 0.4386 - crf_marginal_accuracy: 0.9969 - val_loss: 0.4468 - val_crf_marginal_accuracy: 0.9960
Epoch 5/30
857/857 [==============================] - 91s 106ms/step - loss: 0.4189 - crf_marginal_accuracy: 0.9970 - val_loss: 0.4338 - val_crf_marginal_accuracy: 0.9970
Epoch 6/30
857/857 [==============================] - 91s 106ms/step - loss: 0.3850 - crf_marginal_a

In [42]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.9026    0.8602    0.8809      1109
I-GoodsString     0.8842    0.8487    0.8661      1718

    micro avg     0.8914    0.8532    0.8719      2827
    macro avg     0.8934    0.8544    0.8735      2827
 weighted avg     0.8914    0.8532    0.8719      2827



## W2V(fixed, window=5, min_count=2) + BILSTM + CRF

In [47]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 50

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 3004, 300)         2544900   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 3004, 100)         140400    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 3004, 3)           303       
_________________________________________________________________
crf_14 (CRF)                 (None, 3004, 3)           27        
Total params: 2,685,630
Trainable params: 140,730
Non-trainable params: 2,544,900
_________________________________________________________________


In [48]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='not_pretrained_best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('not_pretrained_best_model_fixed_embeds.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 94s 110ms/step - loss: 0.8406 - crf_marginal_accuracy: 0.7011 - val_loss: 0.6911 - val_crf_marginal_accuracy: 0.9962
Epoch 2/30
857/857 [==============================] - 86s 100ms/step - loss: 0.6389 - crf_marginal_accuracy: 0.9966 - val_loss: 0.5791 - val_crf_marginal_accuracy: 0.9962
Epoch 3/30
857/857 [==============================] - 86s 100ms/step - loss: 0.5386 - crf_marginal_accuracy: 0.9966 - val_loss: 0.5046 - val_crf_marginal_accuracy: 0.9962
Epoch 4/30
857/857 [==============================] - 85s 100ms/step - loss: 0.4817 - crf_marginal_accuracy: 0.9966 - val_loss: 0.4678 - val_crf_marginal_accuracy: 0.9962
Epoch 5/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4562 - crf_marginal_accuracy: 0.9965 - val_loss: 0.4586 - val_crf_marginal_accuracy: 0.9956
Epoch 6/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4531 - crf_marginal_a

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.0000    0.0000    0.0000      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.0000    0.0000    0.0000      2827
    macro avg     0.0000    0.0000    0.0000      2827
 weighted avg     0.0000    0.0000    0.0000      2827



## W2V(not fixed, window=5, min_count=1) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=5, min_count=1)
for d in all_dfs:
    d['int_tokens'] = d.apply(find_token, axis=1)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3004, 300)         3708300   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 3004, 200)         320800    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 3004, 3)           603       
_________________________________________________________________
crf_6 (CRF)                  (None, 3004, 3)           27        
Total params: 4,029,730
Trainable params: 4,029,730
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 93s 109ms/step - loss: 0.5392 - crf_marginal_accuracy: 0.0013 - val_loss: 0.4942 - val_crf_marginal_accuracy: 0.0016
Epoch 2/30
857/857 [==============================] - 89s 104ms/step - loss: 0.4665 - crf_marginal_accuracy: 0.1112 - val_loss: 0.4630 - val_crf_marginal_accuracy: 0.9959
Epoch 3/30
857/857 [==============================] - 89s 104ms/step - loss: 0.4543 - crf_marginal_accuracy: 0.9965 - val_loss: 0.4590 - val_crf_marginal_accuracy: 0.9963
Epoch 4/30
857/857 [==============================] - 88s 103ms/step - loss: 0.4498 - crf_marginal_accuracy: 0.9941 - val_loss: 0.4581 - val_crf_marginal_accuracy: 0.9902
Epoch 5/30
857/857 [==============================] - 88s 103ms/step - loss: 0.4469 - crf_marginal_accuracy: 0.9952 - val_loss: 0.4561 - val_crf_marginal_accuracy: 0.9966
Epoch 6/30
857/857 [==============================] - 88s 103ms/step - loss: 0.4417 - crf_marginal_a

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.9130    0.8512    0.8810      1109
I-GoodsString     0.8340    0.8335    0.8338      1718

    micro avg     0.8637    0.8405    0.8519      2827
    macro avg     0.8735    0.8424    0.8574      2827
 weighted avg     0.8650    0.8405    0.8523      2827



## W2V(fixed, window=5, min_count=1) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 89s 104ms/step - loss: 0.6836 - crf_marginal_accuracy: 0.0029 - val_loss: 0.6254 - val_crf_marginal_accuracy: 0.0018
Epoch 2/30
857/857 [==============================] - 85s 99ms/step - loss: 0.5899 - crf_marginal_accuracy: 0.0014 - val_loss: 0.5625 - val_crf_marginal_accuracy: 0.0014
Epoch 3/30
857/857 [==============================] - 85s 99ms/step - loss: 0.5366 - crf_marginal_accuracy: 0.0013 - val_loss: 0.5185 - val_crf_marginal_accuracy: 0.0014
Epoch 4/30
857/857 [==============================] - 85s 99ms/step - loss: 0.4988 - crf_marginal_accuracy: 0.0013 - val_loss: 0.4886 - val_crf_marginal_accuracy: 0.0014
Epoch 5/30
857/857 [==============================] - 85s 100ms/step - loss: 0.4741 - crf_marginal_accuracy: 0.0013 - val_loss: 0.4707 - val_crf_marginal_accuracy: 0.0014
Epoch 6/30
857/857 [==============================] - 85s 99ms/step - loss: 0.4603 - crf_marginal_accur

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.0000    0.0000    0.0000      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.0000    0.0000    0.0000      2827
    macro avg     0.0000    0.0000    0.0000      2827
 weighted avg     0.0000    0.0000    0.0000      2827



## W2V(not fixed, window=5, max_vocab_size=15000) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=5, max_vocab_size=15000)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 3004, 300)         1419600   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 3004, 200)         320800    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 3004, 3)           603       
_________________________________________________________________
crf_8 (CRF)                  (None, 3004, 3)           27        
Total params: 1,741,030
Trainable params: 1,741,030
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 135s 158ms/step - loss: 0.5672 - crf_marginal_accuracy: 0.0013 - val_loss: 0.5213 - val_crf_marginal_accuracy: 0.0014
Epoch 2/30
857/857 [==============================] - 125s 146ms/step - loss: 0.4992 - crf_marginal_accuracy: 0.0013 - val_loss: 0.4927 - val_crf_marginal_accuracy: 0.0014
Epoch 3/30
857/857 [==============================] - 126s 147ms/step - loss: 0.4757 - crf_marginal_accuracy: 0.0013 - val_loss: 0.4734 - val_crf_marginal_accuracy: 0.0014
Epoch 4/30
857/857 [==============================] - 125s 146ms/step - loss: 0.4607 - crf_marginal_accuracy: 0.0017 - val_loss: 0.4626 - val_crf_marginal_accuracy: 0.0024
Epoch 5/30
857/857 [==============================] - 125s 146ms/step - loss: 0.4538 - crf_marginal_accuracy: 0.0026 - val_loss: 0.4587 - val_crf_marginal_accuracy: 0.0028
Epoch 6/30
857/857 [==============================] - 122s 142ms/step - loss: 0.4520 - crf_marg

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.0000    0.0000    0.0000      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.0000    0.0000    0.0000      2827
    macro avg     0.0000    0.0000    0.0000      2827
 weighted avg     0.0000    0.0000    0.0000      2827



## W2V(fixed, window=5, max_vocab_size=15000) + BILSTM + CRF

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 3004, 300)         1419600   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 3004, 200)         320800    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 3004, 3)           603       
_________________________________________________________________
crf_10 (CRF)                 (None, 3004, 3)           27        
Total params: 1,741,030
Trainable params: 321,430
Non-trainable params: 1,419,600
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 92s 107ms/step - loss: 0.5790 - crf_marginal_accuracy: 0.0019 - val_loss: 0.4979 - val_crf_marginal_accuracy: 0.0080
Epoch 2/30
857/857 [==============================] - 85s 99ms/step - loss: 0.4748 - crf_marginal_accuracy: 0.5495 - val_loss: 0.4782 - val_crf_marginal_accuracy: 0.9737
Epoch 3/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4612 - crf_marginal_accuracy: 0.9812 - val_loss: 0.4644 - val_crf_marginal_accuracy: 0.9859
Epoch 4/30
857/857 [==============================] - 85s 99ms/step - loss: 0.4527 - crf_marginal_accuracy: 0.8922 - val_loss: 0.4603 - val_crf_marginal_accuracy: 0.9858
Epoch 5/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4495 - crf_marginal_accuracy: 0.9893 - val_loss: 0.4592 - val_crf_marginal_accuracy: 0.9932
Epoch 6/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4472 - crf_marginal_acc

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.2814    0.2687    0.2749      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.2166    0.1054    0.1418      2827
    macro avg     0.1407    0.1344    0.1375      2827
 weighted avg     0.1104    0.1054    0.1078      2827



## W2V(not fixed, window=25) + BILSTM + CRF

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=300, window=25, min_count=2)

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 3004, 300)         2544900   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 3004, 200)         320800    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 3004, 3)           603       
_________________________________________________________________
crf_11 (CRF)                 (None, 3004, 3)           27        
Total params: 2,866,330
Trainable params: 2,866,330
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 97s 113ms/step - loss: 0.7928 - crf_marginal_accuracy: 0.0043 - val_loss: 0.7336 - val_crf_marginal_accuracy: 0.0073
Epoch 2/30
857/857 [==============================] - 90s 105ms/step - loss: 0.6980 - crf_marginal_accuracy: 0.0069 - val_loss: 0.6589 - val_crf_marginal_accuracy: 0.0120
Epoch 3/30
857/857 [==============================] - 89s 104ms/step - loss: 0.6320 - crf_marginal_accuracy: 0.0085 - val_loss: 0.6064 - val_crf_marginal_accuracy: 0.0101
Epoch 4/30
857/857 [==============================] - 89s 104ms/step - loss: 0.5851 - crf_marginal_accuracy: 0.0082 - val_loss: 0.5657 - val_crf_marginal_accuracy: 0.0108
Epoch 5/30
857/857 [==============================] - 89s 104ms/step - loss: 0.5470 - crf_marginal_accuracy: 0.0079 - val_loss: 0.5319 - val_crf_marginal_accuracy: 0.0100
Epoch 6/30
857/857 [==============================] - 89s 104ms/step - loss: 0.5160 - crf_marginal_a

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.0000    0.0000    0.0000      1109
I-GoodsString     0.0379    1.0000    0.0730      1718

    micro avg     0.0379    0.6077    0.0713      2827
    macro avg     0.0189    0.5000    0.0365      2827
 weighted avg     0.0230    0.6077    0.0444      2827



## GLOVE (not fixed) + BILSTM + CRF

In [6]:
from glove import Corpus, Glove

#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(df['tokens'], window=5)

glove = Glove(no_components=50, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [9]:
def find_glove_token(x):
    res = []
    for w in x['tokens']:
        if w in glove.dictionary.keys():
            res.append(glove.dictionary[w])
        else:
            res.append(0)
    return res

for d in all_dfs:
    d['encoded_ner_tags'] = d.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
    d['int_tokens'] = d.apply(find_glove_token, axis=1)

In [12]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 50

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 3004, 50)          618050    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 3004, 100)         40400     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3004, 3)           303       
_________________________________________________________________
crf_2 (CRF)                  (None, 3004, 3)           27        
Total params: 658,780
Trainable params: 658,780
Non-trainable params: 0
_________________________________________________________________


In [13]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='not_pretrained_glove_best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('not_pretrained_glove_best_model.h5')




Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 89s 104ms/step - loss: 0.6272 - crf_marginal_accuracy: 0.9930 - val_loss: 0.4888 - val_crf_marginal_accuracy: 0.9963
Epoch 2/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4588 - crf_marginal_accuracy: 0.9936 - val_loss: 0.4634 - val_crf_marginal_accuracy: 0.9964
Epoch 3/30
857/857 [==============================] - 86s 100ms/step - loss: 0.4333 - crf_marginal_accuracy: 0.9970 - val_loss: 0.4511 - val_crf_marginal_accuracy: 0.9967
Epoch 4/30
857/857 [==============================] - 86s 100ms/step - loss: 0.3979 - crf_marginal_accuracy: 0.9973 - val_loss: 0.4313 - val_crf_marginal_accuracy: 0.9969
Epoch 5/30
857/857 [==============================] - 86s 100ms/step - loss: 0.3423 - crf_marginal_accuracy: 0.9977 - val_loss: 0.4080 - val_crf_marginal_accuracy: 0.9973
Epoch 6/30
857/857 [==============================] - 88s 102ms/step - loss: 0.2862 - crf_margina

In [15]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

               precision    recall  f1-score   support

B-GoodsString     0.9172    0.8693    0.8926      1109
I-GoodsString     0.8951    0.8737    0.8842      1718

    micro avg     0.9036    0.8719    0.8875      2827
    macro avg     0.9061    0.8715    0.8884      2827
 weighted avg     0.9037    0.8719    0.8875      2827



## GLOVE (fixed) + BILSTM + CRF

In [16]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 50

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.01), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3004, 50)          618050    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 3004, 100)         40400     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3004, 3)           303       
_________________________________________________________________
crf_3 (CRF)                  (None, 3004, 3)           27        
Total params: 658,780
Trainable params: 40,730
Non-trainable params: 618,050
_________________________________________________________________


In [17]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='not_pretrained_glove_best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('not_pretrained_glove_best_model_fixed_embeds.h5')

Train on 857 samples, validate on 96 samples
Epoch 1/30
857/857 [==============================] - 84s 98ms/step - loss: 0.4954 - crf_marginal_accuracy: 0.9474 - val_loss: 0.4703 - val_crf_marginal_accuracy: 0.9340
Epoch 2/30
857/857 [==============================] - 82s 96ms/step - loss: 0.4561 - crf_marginal_accuracy: 0.9475 - val_loss: 0.4598 - val_crf_marginal_accuracy: 0.9961
Epoch 3/30
857/857 [==============================] - 82s 96ms/step - loss: 0.4555 - crf_marginal_accuracy: 0.9966 - val_loss: 0.4620 - val_crf_marginal_accuracy: 0.9962
Epoch 4/30
857/857 [==============================] - 82s 96ms/step - loss: 0.4546 - crf_marginal_accuracy: 0.9966 - val_loss: 0.4588 - val_crf_marginal_accuracy: 0.9962
Epoch 5/30
857/857 [==============================] - 82s 96ms/step - loss: 0.4520 - crf_marginal_accuracy: 0.9838 - val_loss: 0.4594 - val_crf_marginal_accuracy: 0.9255
Epoch 6/30
857/857 [==============================] - 82s 96ms/step - loss: 0.4524 - crf_marginal_accurac

In [18]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
named_preds = []
for i in truncated_preds:
    named_preds.append([])
    for j in i:
        named_preds[-1].append(label_list[j])

print(metrics.flat_classification_report(test['ner_tags'], named_preds, digits=4, labels=labels))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

B-GoodsString     0.0000    0.0000    0.0000      1109
I-GoodsString     0.0000    0.0000    0.0000      1718

    micro avg     0.0000    0.0000    0.0000      2827
    macro avg     0.0000    0.0000    0.0000      2827
 weighted avg     0.0000    0.0000    0.0000      2827

